In [53]:
!pip install autocorrect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import pandas as pd
import json
import os
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import json
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import time
from tabulate import tabulate
from autocorrect import Speller

In [55]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [56]:
f=open('drive/My Drive/AIR/inverted_index1.json','r')
inverted_index=json.loads(f.read())
f=open('drive/My Drive/AIR/record_list.json','r')
record_list=json.loads(f.read())
f=open('drive/My Drive/AIR/permuterm1.json','r')
permuterm_index=json.loads(f.read())
f=open('drive/My Drive/AIR/snippet_df.txt','r')
snippet_df=eval(f.read())
doc_location='drive/My Drive/archive/TelevisionNews/'

In [57]:
nltk.download('stopwords')
stopWords = stopwords.words("english")
stopWords.append("i\'ve")
stopWords.append("i\'m")
stopWords.append("he\'d")
stopWords.append("she\'d")
stopWords.append("i\'d")
stopWords.append("i\'ll")
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:          
        return None

def lemmatize_sentence(sentence):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word in sentence.split():
            lemmatized_sentence.append(lemmatizer.lemmatize(word))
    return " ".join(lemmatized_sentence)

def special_characters_removal(word):
  special=list(string.punctuation)
  special.append(' ')
  special.remove('*')
  x=[]
  for i in word:
    if i not in special:
      x.append(i)
    else:
      x.append(' ')
  return ''.join(x) 

def preprocessing(snippet):
  snippet=snippet.split()
  res=[]
  for i in range(len(snippet)):
    if snippet[i] not in stopWords and len(snippet[i])>1:
      res.extend(special_characters_removal(snippet[i]).split())
  snippet=" ".join(res)
  snippet=lemmatize_sentence(snippet)
  return snippet.lower()  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## **`Boolean Queries`**


In [58]:
def mergeand(left_operand,right_operand):
  doc_list=set(left_operand.keys()).intersection(set(right_operand.keys()))
  final_record_list={}
  for k in doc_list:
    x=left_operand[k].intersection(right_operand[k])
    if x:
      final_record_list[k]=x
  return final_record_list

def mergeor(left_operand,right_operand):
  doc_list=set(left_operand.keys()).union(set(right_operand.keys()))
  final_record_list={k:left_operand[k].union(right_operand[k] if k in right_operand else set()) if k in left_operand else right_operand[k] for k in doc_list}
  return final_record_list

def get_not_posting_list(operand):
  files=set(record_list.keys())
  post_list={k:set(map(str,range(record_list[k][0]))).difference(operand[k] if k in operand else {}) for k in files}
  return post_list

def show_records(final):
  result=[]
  for i in final:
    doc_l=doc_location+i
    df=pd.read_csv(doc_l)
    for j in final[i]:
      result.append(df.loc[int(j)])
  return result

def get_posting_list(term):
  if term not in inverted_index:
    return dict()
  x=list(inverted_index[term].keys())
  x.remove('doc_frequency')
  post_list={k:set(inverted_index[term][k]['records'].keys()) for k in x}
  return post_list


In [59]:
def parsebooleanquery(query):
  query=query.replace('(',' ( ')
  query=query.replace(')',' ) ')
  precedence={}
  precedence['NOT']=3
  precedence['OR']=2
  precedence['AND']=1
  precedence['(']=0
  precedence[')']=0
  operand_stack=[]
  operator_stack=[]
  postfix_stack=[]
  j=0
  l=query.split()
  while(j<len(l)):
    i=l[j]
    if i=="(":
      operator_stack.append(i)
    elif i==")":
      while( operator_stack and operator_stack[-1]!="(" ):
        postfix_stack.append(operator_stack.pop())
      operator_stack.pop()
    elif i in precedence:
      while(operator_stack and precedence[operator_stack[-1]]>precedence[i]):
        postfix_stack.append(operator_stack.pop())
      operator_stack.append(i)
    else:
      x=i
      j+=1
      while(j<len(l) and l[j] not in precedence):
        x=x+" "+l[j]
        j+=1
      if not operator_stack or operator_stack[-1]!="NOT":
        operand_stack.append(x)
      postfix_stack.append(x)
      j-=1
    j+=1
  while(operator_stack):
    postfix_stack.append(operator_stack.pop())
  return postfix_stack,operand_stack

def eval_query(postfix_stack):
  result=[]
  for i in postfix_stack:
    if i=="AND":
      left_operand=result.pop()
      right_operand=result.pop()
      result.append(mergeand(left_operand,right_operand))
    elif i=="OR":
      left_operand=result.pop()
      right_operand=result.pop()
      result.append(mergeor(left_operand,right_operand))
    elif i=="NOT":
      operand=result.pop()
      result.append(get_not_posting_list(operand))
    else:
      result.append(parsephrasequery(i) if '*' not in i else eval_wildcard_query(i))
  return result[0]

def get_results(query):
  p_stack,o_stack=parsebooleanquery(query)
  return eval_query(p_stack),o_stack

# Phrase query

In [60]:
def parsephrasequery(query):
  query=preprocessing(spellcheck(query))
  tokens=query.split()
  intermediate_result=get_posting_list(tokens[0])
  for i in range(1,len(tokens)):
    intermediate_result=mergeand(intermediate_result,get_posting_list(tokens[i]))
  final_record_list=mergephrase(tokens,intermediate_result) if len(tokens)>1 else intermediate_result
  return final_record_list

def get_positional_list(doc_name, record_no, term):
  return inverted_index[term][doc_name]['records'][record_no]['position'] 

def mergephrase(tokens,intersect_list):
  final_record_list={}
  for doc in intersect_list:
    records=intersect_list[doc]
    for record in records:
      temp_pos=[]
      for i in range(len(tokens)):
        temp_pos.append(set(map(lambda x:x-i,get_positional_list(doc, record, tokens[i]))))
      if (set(temp_pos[0].intersection(*temp_pos))):
        if (doc not in final_record_list):
          final_record_list[doc]=set()
        final_record_list[doc].add(record)
  return final_record_list  

# Wildcard Queries

In [61]:
def splitquery(query):
  query1=query.split("*")
  if query1[0]=='' and len(query1)==3 and query1[2]=='':
    return [query]
  elif len(query1)==2 and (query1[0]=='' or  query1[1]==''):
    return [query]
  else:
    result=[]
    if query1[0]=='':
      result.append("*"+query1[1]+"*")
      i=2
    else:
      result.append(query1[0]+"*")
      i=1
    while(i<len(query1)-1):
      result.append("*"+query1[i]+"*")
      i+=1
    if query1[i]!='':
      result.append("*"+query1[i])
    return result
  
def permuterm_form(query):
  query=query.split('*')
  if query[0]=='' and len(query)==3 and query[2]=='': #*X*
    actual_query=query[1]
  elif query[0]=='' and len(query)==2:  #*X
    actual_query=query[1]+"$"
  elif query[1]=='' and len(query)==2:  #X*
    actual_query="$"+query[0]
  return actual_query

def parsewildcardquery(input_query):
  x=splitquery(input_query)
  result=get_wildcard_result(permuterm_form(x[0]))
  for query in x[1:]:
    result=result.intersection(get_wildcard_result(permuterm_form(query)))
  tokens=input_query.split('*')
  result1=[]
  for i in result:
    y=i
    flag=0
    for j in tokens:
      if j=='':
        pass
      else:
        pos=y.find(j)
        if pos==-1:
          flag=1
          break
        else:
          y=y[pos+len(j):]
    if flag==0:
      result1.append(i)
  return result1

def get_wildcard_result(query):
  result=[]
  for i in permuterm_index:
    if i.startswith(query):
      result.append(permuterm_index[i])
  return set(result)

def eval_wildcard_query(query):
  x=parsewildcardquery(query)
  result=get_posting_list(x[0])
  for i in x[1:]:
    result=mergeor(get_posting_list(i),result)
  return result

# Ranking

In [62]:
import nltk
nltk.download('omw-1.4')
preprocessed_snippet_df=list(map(lambda x:preprocessing(x[0]),snippet_df))

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [63]:
def ranked_results(query,k):
  f=open('text2.txt','w')
  t1=time.time()
  results,tokens=get_results(query)
  if not results:
    return "No results obtained."
  tokens1=[]
  for i in range(len(tokens)):
    if '*' in tokens[i]:
      tokens1.extend(list(parsewildcardquery(tokens[i])))
    else:
      tokens1.append(tokens[i])
  filtered_tfidf=[]
  snippets=[]
  for i in results:
    for j in results[i]:
      filtered_tfidf.append(record_list[i][1]+int(j))
      snippets.append(preprocessed_snippet_df[filtered_tfidf[-1]])
  results1=[]
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(snippets)
  query_vec = vectorizer.transform([preprocessing(" ".join(tokens1))])
  results1=cosine_similarity(X,query_vec).reshape((-1,))
  results2=sorted(range(len(results1)),reverse=True, key=lambda x:results1[x])
  t2=time.time()
  print()
  k=min(k,len(results2))
  print(len(results2)," results fetched in ",t2-t1,"seconds. Top ",k,"are being shown below - ")
  print()
  table=[]
  for i in results2[:k]:
      x=snippet_df[filtered_tfidf[i]]
      table.append([x[1],x[2],x[0]])
  print(tabulate(table,headers=["Document Name","Row number","Snippet"]))
  print()
  for i in results2:
      f.write(snippet_df[filtered_tfidf[i]][0])
      f.write("\n")
  f.close()

# Spelling correction

In [64]:
def spellcheck(query):
  spell = Speller(lang='en')
  query1=query
  query = query.split()
  corrected=[]
  for i in range(len(query)):
    if query[i] in inverted_index:
      corrected.append(query[i])
    else:
      corrected.append(spell(query[i]))
  corrected=" ".join(corrected)
  if corrected!=query1:
    print("Did you mean",corrected,"?")
  return corrected

# Mixed Queries

In [67]:
ranked_results('hello',10)


280  results fetched in  0.10031819343566895 seconds. Top  10 are being shown below - 

Document Name         Row number  Snippet
------------------  ------------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
BBCNEWS.201806.csv            59  to tackle climate change and air pollution hello, everyone - this is afternoon live.
MSNBC.201811.csv             182  you, too. and ahead, what those wildfires in california are telling us. behind the lost lives and burnt forest is the growing threat affect being us all, climate change. hello!. hello is friendly.
BBCNEWS.201808.csv            77  climate change, making parts of the planet uninhabitable. hello, everyone, this is afternoon live.
MSNBC.201612.csv             208  